In [1]:
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import numpy as np
import ipywidgets as widgets
import scipy.integrate as integrate

from ipywidgets import interact, fixed
from scipy.interpolate import interp1d
from textwrap import dedent
from bokeh.io import push_notebook, show, output_notebook, output_file, curdoc
from bokeh.plotting import figure
from bokeh.layouts import column, row
from bokeh.themes import built_in_themes
from bokeh.models import Range1d, BasicTickFormatter, Label, Legend
from IPython.display import IFrame

# plot bokeh in notebook format
output_notebook()
curdoc().theme = 'dark_minimal'

Loading BokehJS ...

In [2]:
# define constants
kB = 1.380649e-23 # J/K
hbar = 6.626e-34 / (2*np.pi) # J*s
e = 1.60217733e-19 # C
m0 = 9.109383e-31 # kg
T = 300 # K
epsilon0 = 8.854187e-12 # F/m
kBeV = kB/e
Nmin = 1e16; Nmax = 1e18 # cm^-3

# define semiconductor properties
semiconductorList = ['Si', 'Ge','GaAs']
Eg_ = np.array([1.11, 0.66, 1.43]) # Bandgap / eV
me_ = np.array([1.09, 0.55, 0.067]) # average effective electron mass
mh_ = np.array([1.15, 0.37, 0.45]) # average effective hole mass
epsilonR_ = np.array([11.9, 16.2, 13.1]) # relative permittivity


In [8]:
class pn(object):
    # initializer 
    def __init__(self, semiconductor = 'Si'):
        self.semiconductor = semiconductor
        self.nd = 4e17
        self.na = 5e17
    
    # change pn junction type and get associated parameters
    def set_semiconductor(self, semiconductor):
        i = semiconductorList.index(semiconductor)
        self.eg, self.me, self.mh, self.epsilon = Eg_[i], me_[i]*m0, mh_[i]*m0, epsilonR_[i]*epsilon0
        return "Bandgap: {} eV".format(self.eg)
    
    # set p & n doping   
    def set_doping(self, Nd, Na):
        self.nd = Nd
        self.na = Na
        
    # calculate pn-junction properties depending on bandgap and doping
    def get_properties(self):
        self.nc = 1/4* ( 2*self.me*kB*T / (np.pi*hbar**2) )**(3/2) *1e-6 # cm^-3
        self.nv = 1/4* ( 2*self.mh*kB*T / (np.pi*hbar**2) )**(3/2) *1e-6 # cm^-3

        self.ni = (self.nc*self.nv)**(1/2) * np.exp(-self.eg / (2*kBeV*T))

        self.e_Delta_phi = ( self.eg + kBeV*T*np.log(self.nd*self.na / (self.nc*self.nv)) )
        
        # equation for dn, dn length
        def dnp(doping = 'n'):
            if doping == 'n': exponent = 1;
            elif doping == 'p': exponent = -1;
             # factor 1e6 for cm^-3, 1e10 for Angstrom
            return ( (self.na/self.nd)**exponent / ((self.na+self.nd) *1e6) * 2 
                   * self.epsilon * self.e_Delta_phi / e )**(1/2) * 1e10

        self.dn = dnp(doping = 'n')
        self.dp = dnp(doping = 'p')
        return (self.dp, self.dn, self.na, self.nd)

    
    # calculate carrier density
    def carrier_density(self, x):
        return np.piecewise(x,  [x < -self.dp-self.dn, 
                             ((-self.dp-self.dn <= x) & (x < -self.dn)), 
                             ((-self.dn <= x) & (x < 0)), 
                             ((0 <= x) & (x <= self.dn)), 
                             ((self.dn < x) & (x <= self.dn + self.dn))],
                            [self.na, 0, self.nd, self.nd, 0, self.na])

    # calculate charge density
    def charge_density(self, x):
        return np.piecewise(x, [x < -self.dp-self.dn, 
                             ((-self.dp-self.dn <= x) & (x < -self.dn)), 
                             ((-self.dn <= x) & (x < -self.dn / 4)), (x > -self.dn / 4) & (x<= self.dn / 4), 
                             ((self.dn / 4 <= x) & (x <= self.dn)), 
                             ((self.dn < x) & (x <= self.dn + self.dn))],
                            [0, -self.na, self.nd, 0, self.nd, -self.na, 0])

    # calculate potential
    def phi1(self, x):
        return (2*np.pi*e * self.na*1e6 / self.epsilon) * (((x + self.dp + self.dn)/1e10)**2 - (self.dp/1e10)**2)
    def phi2(self, x):
        return (2*np.pi*e * self.nd*1e6 / self.epsilon) * ((self.dn/1e10)**2 - ((x)/1e10)**2)
    def phi3(self, x):
        return (2*np.pi*e * self.nd*1e6 / self.epsilon) * ((self.dn/1e10)**2 - ((-x)/1e10)**2)
    def phi4(self, x):
        return (2*np.pi*e * self.na*1e6 / self.epsilon) * ((((-x + self.dp + self.dn))/1e10)**2 - (self.dp/1e10)**2)

        return np.piecewise(x, [x < -self.dp, ((-self.dp <= x) & (x < 0)), ((0 <= x) & (x <= self.dn)), self.dn < x],
                            [self.phi1(x = -self.dp), self.phi1, self.phi2, self.phi2(x = self.dn)]) / self.epsilon*epsilon0 
    
    # calculate potential
    def potential(self, x):
        return np.piecewise(x, 
                            [x < -self.dp-self.dn, 
                             ((-self.dp-self.dn <= x) & (x < -self.dn)), 
                             ((-self.dn <= x) & (x < 0)), 
                             ((0 <= x) & (x <= self.dn)), 
                             ((self.dn < x) & (x <= self.dp + self.dn)), 
                             x > self.dp+self.dn],
                            [self.phi1(x = -self.dp-self.dn), 
                             self.phi1, 
                             self.phi2, 
                             self.phi3, 
                             self.phi4, 
                             self.phi4(x = self.dp+self.dn)]) / self.epsilon * epsilon0
# create pn-junction object
pn1 = pn()
pn1.set_semiconductor(pn1.semiconductor);

In [9]:
# define 1D axis along pn-junction
a = 3; N = 2**9+ 1
dp, dn, na, nd = pn1.get_properties()
xMax = np.maximum(dn, dp)*a
x = np.linspace(-xMax,xMax, N)

# plot carrier & charge densities, potential
def plot_analytic(pnx, x):
    # create the plots
    kwarsFig = {'width':800, 'height': 200}
    s1 = figure(title = 'Carrier density',y_range=(0, Nmax), **kwarsFig)
    s1.toolbar.autohide = True
    s1.yaxis.axis_label = 'cm^-3'
    s1.yaxis.formatter = BasicTickFormatter(precision = 0)
    kwarsLabel = {'y': 6e17, 'text_font_style': 'bold', 'text_align': 'center', 'text_color': '#226baa'}
    s1.add_layout(Label(text = "p-type", x = -1000, **kwarsLabel))
    s1.add_layout(Label(text = "n-type", x = 0, **kwarsLabel))
    s1.add_layout(Label(text = "p-type", x = 1000, **kwarsLabel))
    r1 = s1.line(x, pnx.carrier_density(x),line_width = 3)
    
    s2 = figure(title = 'Charge density',y_range=(-Nmax, Nmax), **kwarsFig)
    kwarsLabelTop = {'text_font_style': 'bold', 'text_align': 'center', 'text_color': '#226baa'}
    kwarsLabelBottom = {'text_font_style': 'bold', 'text_align': 'center', 'text_color': '#226baa'}
    s2.add_layout(Label(text="n-type", x=-300, y=1000, **kwarsLabelTop))
    s2.add_layout(Label(text="n-type", x=300, y=1000, **kwarsLabelTop))
    s2.add_layout(Label(text="p-type", x=-500, y=-Nmax + 100, **kwarsLabelBottom))
    s2.add_layout(Label(text="p-type", x=500, y=-Nmax + 100, **kwarsLabelBottom))
    s2.toolbar.autohide = True
    s2.yaxis.formatter = BasicTickFormatter(precision = 0)
    r2 = s2.line(x, pnx.charge_density(x),line_width = 3)
    s2.yaxis.axis_label = 'cm^-3'
    
    s3 = figure(title = 'Potential',y_range=(-1.1, 1.1), **kwarsFig)
    s3.toolbar.autohide = True
    r3 = s3.line(x, pnx.potential(x),line_width = 3)
    s3.xaxis.axis_label = 'x [Å]'
    s3.yaxis.axis_label = 'V'
    
    s = [s1,s2,s3]; r = [r1,r2,r3]
    
    for sx in s:
        sx.x_range = Range1d(-xMax, xMax)
    return s, r
    
def plot_update(pnx, r, x, handle):
    r[0].data_source.data['y'] = pnx.carrier_density(x)
    r[1].data_source.data['y'] = pnx.charge_density(x)
    r[2].data_source.data['y'] = pnx.potential(x)
    push_notebook(handle=handle)
    return

# show plots
s, r = plot_analytic(pn1, x)
handle1 = show(column(s), notebook_handle=True);
plot_update(pn1, r,x, handle1)

In [10]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, widgets
%matplotlib widget

# Set style manually to mimic 'dark_minimal'
plt.rcParams['figure.facecolor'] = '#2F2F2F'
plt.rcParams['axes.facecolor'] = '#2F2F2F'
plt.rcParams['figure.edgecolor'] = '#2F2F2F'
plt.rcParams['axes.edgecolor'] = '#2F2F2F'
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.5
plt.rcParams['grid.color'] = "#ffffff"
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False
plt.rcParams['xtick.color'] = '#ffffff'
plt.rcParams['ytick.color'] = '#ffffff'
plt.rcParams['axes.labelcolor'] = '#ffffff'
plt.rcParams['axes.titlecolor'] = '#ffffff'
plt.rcParams['lines.color'] = '#1f77b4'  # Similar to bokeh line color

# constants
Is = 1e-12

Ve = 0 # you can change this value
plt.figure(figsize=(10, 6))

def transistor_current(Vc, Vb):
    if Vb == "On":
        Vb = 0.01
    else:
        Vb = 0
    return Is * (np.exp(Vc - Vb) - 1) * (np.exp(Vb - Ve) - 1)

# interactive plot
@interact(Vb=widgets.ToggleButtons(options=["Off", "On"],labels=["Off","On"], description='Base Voltage:'))
def plot(Vb):
    Vc = np.linspace(0, 2, 400)
    I = transistor_current(Vc, Vb)
    
    plt.clf()
    plt.plot(Vc, I)
    plt.ylim(-1e-14,6e-14)
    plt.xlim(0,2)
    plt.title('Transistor: I for Vc')
    plt.xlabel('Vc [В]')
    plt.ylabel('I [А]')
    plt.grid(True, which='both', color='white', linewidth=0.5, alpha=0.5)
    plt.show()


interactive(children=(ToggleButtons(description='Base Voltage:', options=('Off', 'On'), value='Off'), Output()…

## Transistor Current Explanation

This formula models the current through a **bipolar junction transistor (BJT)**. This transistor has three terminals: the **emitter**, the **base**, and the **collector**. The voltage across these terminals is denoted by `Ve`, `Vb`, and `Vc`, respectively. `Is` is the scale current (or saturation current), which is a small leakage current that exists even when the transistor is off.

The expression `np.exp(Vc - Vb) - 1` models the current between the **collector** and the **base**. Similarly, `np.exp(Vb - Ve) - 1` models the current between the **base** and the **emitter**. Multiplying these two quantities together gives the overall transistor current. 

### Why are there exponentials?

The presence of the exponential functions in the formula is due to the nature of the **p-n junctions** that are integral to the structure of a transistor. A p-n junction, the basic building block of many semiconductor electronic devices, exhibits exponential I-V characteristics (current-voltage). The exponential form arises because of the diffusion of carriers (holes and electrons) across the junction and their recombination. The larger the applied voltage, the larger the driving force for the carriers to move across the junction, hence the exponential increase in current.

### Why is there multiplication?

The multiplication of these two exponential terms is an approximation of the behavior of the transistor in **active mode**. This approximation assumes that the base-emitter junction is forward-biased, and the base-collector junction is reverse-biased.

When a small base-emitter voltage (`Vb - Ve`) is applied, a lot of charge carriers are injected into the base region. However, the base region is very thin, so only a small fraction of these carriers recombine there. The majority of them reach the base-collector junction (`Vc - Vb`), where they are swept across (because of the reverse bias of this junction) into the collector. This results in a large collector current.

This behavior is modeled by the multiplication of the two exponential terms. The first exponential term represents the probability of a carrier being injected into the base, while the second exponential term represents the probability of the carrier reaching the collector.

**Note**: This is a simplified model. In reality, transistor behavior is more complex and depends on factors like temperature, fabrication process variations, and the presence of higher-order effects at high frequencies or high currents. Nonetheless, this model is a good starting point for understanding and simulating basic transistor operation.
